In [ ]:
# CELDA 1
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Análisis completo de datos de OpenPowerlifting

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import zipfile
from pathlib import Path
from datetime import datetime
import warnings

# (Opcional) Carga remota a BigQuery
USE_BIGQUERY = True

if USE_BIGQUERY:
    from google.cloud import bigquery
    from google.oauth2 import service_account

warnings.filterwarnings('ignore')

# Setup directorios
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

# Crear directorios si no existen
for d in [DATA_DIR, RAW_DIR, PROCESSED_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"📂 Base directory: {BASE_DIR}")
print("✅ Configuración lista para análisis")


In [ ]:
# CELDA 2 (versión mejorada)
# -*- coding: utf-8 -*-
# ---
# Descarga y conversión eficiente a Parquet desde OpenPowerlifting

import pandas as pd
import requests
import zipfile
from pathlib import Path
import os

# Rutas
RAW_DIR = Path("data/raw")
PROCESSED_DIR = Path("data/processed")
ZIP_URL = "https://openpowerlifting.gitlab.io/opl-csv/files/openpowerlifting-latest.zip"
ZIP_PATH = RAW_DIR / "openpowerlifting-latest.zip"
EXTRACT_PATH = RAW_DIR / "extracted"
PARQUET_PATH = PROCESSED_DIR / "powerlifting_raw.parquet"

# Crear carpetas necesarias
RAW_DIR.mkdir(parents=True, exist_ok=True)
EXTRACT_PATH.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

def download_and_convert():
    """Descargar ZIP, extraer CSV y convertir a Parquet."""
    
    if PARQUET_PATH.exists():
        print(f"📦 Archivo Parquet ya existe: {PARQUET_PATH.name} ({round(os.path.getsize(PARQUET_PATH)/1e6, 1)} MB)")
        return PARQUET_PATH

    if not ZIP_PATH.exists():
        print("📥 Descargando datos de OpenPowerlifting...")
        r = requests.get(ZIP_URL, stream=True)
        total = int(r.headers.get('content-length', 0))
        with open(ZIP_PATH, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print("✅ Descarga completada.")
    
    print("🗂️ Extrayendo archivo ZIP...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    
    print("🔍 Buscando archivo CSV...")
    csv_files = list(EXTRACT_PATH.glob("**/*.csv"))
    main_csv = [f for f in csv_files if 'openpowerlifting' in f.name and f.suffix == '.csv']
    
    if not main_csv:
        raise FileNotFoundError("❌ No se encontró el archivo CSV principal.")
    
    csv_path = main_csv[0]
    print(f"📄 CSV detectado: {csv_path.name}")
    
    print("⚙️ Cargando CSV y convirtiendo a Parquet...")
    df = pd.read_csv(csv_path, low_memory=False)
    
    print(f"✅ Datos cargados: {len(df):,} filas. Guardando como Parquet...")
    df.to_parquet(PARQUET_PATH, index=False)
    print(f"🎉 Parquet guardado en: {PARQUET_PATH} ({round(os.path.getsize(PARQUET_PATH)/1e6, 1)} MB)")

    return PARQUET_PATH

# Ejecutar
parquet_file = download_and_convert()
print(f"📦 Archivo Parquet listo: {parquet_file.name} ({round(os.path.getsize(parquet_file)/1e6, 1)} MB)")

In [ ]:
# CELDA 3 (mejorada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Carga de datos desde archivo Parquet

from pathlib import Path
import pandas as pd

# Ruta al archivo Parquet ya procesado
parquet_path = Path("data/processed/powerlifting_raw.parquet")

# Validar existencia
if not parquet_path.exists():
    raise FileNotFoundError("❌ Archivo Parquet no encontrado. Ejecuta la Celda 2 para generarlo.")

# Cargar datos en memoria eficiente
print("📂 Cargando datos desde archivo Parquet optimizado...")
df = pd.read_parquet(parquet_path)
print(f"✅ Datos cargados correctamente:")
print(f"   - Filas:     {df.shape[0]:,}")
print(f"   - Columnas:  {df.shape[1]:,}")
print(f"   - Columnas disponibles: {', '.join(df.columns[:8])}... (+{len(df.columns)-8} más)" if len(df.columns) > 8 else f"   - Columnas: {df.columns.tolist()}")
print(f"   - Memoria usada: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

In [ ]:
# CELDA 4 (mejorada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Análisis exploratorio inicial

print("📊 INFORMACIÓN BÁSICA DEL DATASET\n" + "-"*40)
print(f"✔️ Filas         : {df.shape[0]:,}")
print(f"✔️ Columnas      : {df.shape[1]:,}")

# Vista preliminar de datos
print("\n🔍 PRIMERAS 3 FILAS:")
try:
    display(df.head(3))  # Jupyter o Streamlit
except Exception:
    print(df.head(3).to_string(index=False))

# Tipos de datos
print("\n🧠 TIPOS DE DATOS:")
print(df.dtypes.value_counts())
print("\nTipos específicos:")
print(df.dtypes.sort_values().astype(str).to_string())

# Valores faltantes
print("\n⚠️ VALORES FALTANTES (TOP 10):")
missing = df.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if not missing.empty:
    print(missing.head(10))
else:
    print("✅ No hay valores faltantes significativos.")

# Estadísticas descriptivas
print("\n📈 ESTADÍSTICAS DESCRIPTIVAS (numéricas):")
print(df.describe(include=[np.number]).T.round(2))

print("\n📋 ESTADÍSTICAS DESCRIPTIVAS (categóricas):")
print(df.describe(include=['object', 'category']).T)
print("\n✅ Análisis exploratorio inicial completado.")

In [ ]:
# CELDA 5 (mejorada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Top atletas por total y países con más registros

# 1. Top 5 atletas con mayores totales
print("🏋️‍♂️ TOP 5 ATLETAS CON TOTALES MÁS ALTOS\n" + "-"*40)
if 'TotalKg' in df.columns and df['TotalKg'].notna().sum() > 0:
    top_athletes = df[df['TotalKg'].notna()].nlargest(5, 'TotalKg')[
        ['Name', 'Sex', 'Equipment', 'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg', 'Country', 'Date']
    ]
    try:
        display(top_athletes)
    except Exception:
        print(top_athletes.to_string(index=False))
else:
    print("❌ No se encontraron registros válidos en 'TotalKg'.")

# 2. Países con más registros
print("\n🌍 TOP 10 PAÍSES CON MÁS REGISTROS\n" + "-"*40)
if 'Country' in df.columns:
    country_counts = df['Country'].value_counts(dropna=True).head(10)
    print(country_counts)
else:
    print("❌ La columna 'Country' no está presente en el dataset.")


In [ ]:
# CELDA 6 (optimizada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Evaluación de calidad y estructura del dataset

print("🔎 ANÁLISIS DE CALIDAD DE DATOS")
print("="*50)

print(f"📄 Dataset original: {len(df):,} filas")
print(f"🧩 Columnas totales: {len(df.columns)}\n")

# Columnas clave esperadas
key_columns = [
    'Name', 'Sex', 'Equipment', 'Age', 'BodyweightKg', 'WeightClassKg',
    'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg',
    'Country', 'Federation', 'Date', 'Dots', 'Wilks'
]
existing_cols = [col for col in key_columns if col in df.columns]

print("📌 COLUMNAS PRINCIPALES PRESENTES:")
for col in existing_cols:
    print(f"  - {col}")
print()

# Valores faltantes (%)
print("🚨 VALORES FALTANTES (%):")
missing = df[existing_cols].isnull().sum()
for col in missing.index:
    nulls = missing[col]
    pct = (nulls / len(df)) * 100
    print(f"  - {col}: {nulls:,} nulos ({pct:.1f}%)")
print()

# Fechas
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    min_date, max_date = df['Date'].min(), df['Date'].max()
    print("📆 RANGO DE FECHAS:")
    print(f"  Desde: {min_date.date() if pd.notna(min_date) else 'N/D'}")
    print(f"  Hasta: {max_date.date() if pd.notna(max_date) else 'N/D'}\n")

# TotalKg
if 'TotalKg' in df.columns:
    print("📊 ESTADÍSTICAS DE 'TotalKg':")
    print(df['TotalKg'].describe(percentiles=[.25, .5, .75]).round(1))
    print()

# WeightClassKg
if 'WeightClassKg' in df.columns:
    print("⚖️ VALORES ÚNICOS EN 'WeightClassKg':")
    values = df['WeightClassKg'].dropna().astype(str).unique().tolist()

    def parse_wclass(x):
        try:
            return float(x.replace("kg", "").replace("+", "").replace(",", ".").strip())
        except:
            return float('inf')

    sorted_values = sorted(values, key=parse_wclass)
    print(f"  Total únicos: {len(sorted_values)}")
    print("  Ejemplos:", sorted_values[:15])

    invalid = df[~df['WeightClassKg'].astype(str).str.contains(r'^\d+(\.\d+)?\+?$', na=False, regex=True)]
    if not invalid.empty:
        print(f"\n⚠️ Valores no estándar detectados en 'WeightClassKg': {len(invalid):,} filas")
else:
    print("❌ No se encontró la columna 'WeightClassKg'")


In [ ]:
# CELDA 7 (optimizada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# LIMPIEZA Y TRANSFORMACIÓN DE DATOS

print("🧹 INICIANDO LIMPIEZA DE DATOS...")
print(f"📊 Filas iniciales: {len(df):,}")

# 1. Filtrar TotalKg > 0
df_clean = df[df['TotalKg'].gt(0)].copy()
print(f"✅ Total > 0: {len(df_clean):,} filas")

# 2. Filtrar fechas desde 1980
df_clean = df_clean[df_clean['Date'] >= pd.to_datetime('1980-01-01')]
print(f"✅ Fechas desde 1980: {len(df_clean):,} filas")

# 3. Filtrar totales realistas
df_clean = df_clean[df_clean['TotalKg'].between(50, 1500)]
print(f"✅ Totales entre 50 y 1500 kg: {len(df_clean):,} filas")

# 4. Estadísticas por categoría
print("\n📊 DISTRIBUCIÓN CATEGÓRICA:")

if 'Sex' in df_clean.columns:
    print("👤 Sexo:")
    print(df_clean['Sex'].value_counts())

if 'Equipment' in df_clean.columns:
    print("\n🏋️ Equipamiento:")
    print(df_clean['Equipment'].value_counts().head(8))

# 5. Columnas derivadas
print("\n🧠 CREANDO COLUMNAS DERIVADAS...")

# Función para clasificar peso corporal
def classify_weight(bw):
    if pd.isna(bw): return "Desconocido"
    try:
        bw = float(bw)
        if bw < 59: return "-59"
        elif bw < 66: return "59-65"
        elif bw < 74: return "66-73"
        elif bw < 83: return "74-82"
        elif bw < 93: return "83-92"
        elif bw < 105: return "93-104"
        elif bw < 120: return "105-119"
        else: return "120+"
    except:
        return "Desconocido"

# Agregar columnas usando assign (más seguro)
df_clean = df_clean.assign(
    Year = df_clean['Date'].dt.year,
    Decade = (df_clean['Date'].dt.year // 10) * 10,
    RelativeStrength = df_clean['TotalKg'] / df_clean['BodyweightKg'].replace({0: np.nan}),
    AgeGroup = pd.cut(
        df_clean['Age'],
        bins=[0, 23, 35, 45, 55, 100],
        labels=['Youth', 'Open', 'Masters1', 'Masters2', 'Masters3+'],
        include_lowest=True
    ),
    WeightClass = df_clean['BodyweightKg'].apply(classify_weight)
)

# Resultado final
print(f"\n🧼 Dataset limpio: {len(df_clean):,} filas")
reduction_pct = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"📉 Reducción total: {reduction_pct:.1f}%")


In [ ]:
# CELDA 8 (optimizada)
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Normalización de nombres de atletas

print("🔤 NORMALIZACIÓN DE NOMBRES DE ATLETAS")
print("="*60)

# -------------------------
# Mostrar ejemplos crudos
# -------------------------
sample_names = df_clean['Name'].dropna().astype(str).unique().tolist()[:30]
print("📌 EJEMPLOS ORIGINALES:")
for i, name in enumerate(sample_names, 1):
    print(f"{i:2d}. '{name}'")

original_unique_names = df_clean['Name'].nunique()
print(f"\n🔢 Nombres únicos (originales): {original_unique_names:,}")

# -------------------------
# Función de normalización
# -------------------------
import unicodedata
import re

def normalize_name(name, remove_accents=False):
    """Normalizar nombres de atletas para análisis y agrupación"""
    if pd.isna(name): return name
    name_clean = str(name).strip().title()
    name_clean = re.sub(r'\s+', ' ', name_clean)                             # Espacios múltiples
    name_clean = re.sub(r"[^\w\s\-\'ÁÉÍÓÚÑáéíóúñ]", '', name_clean)          # Eliminar símbolos raros
    if remove_accents:
        name_clean = ''.join(
            c for c in unicodedata.normalize('NFD', name_clean)
            if unicodedata.category(c) != 'Mn'
        )
    return name_clean

# -------------------------
# Aplicar normalización
# -------------------------
print("\n🔁 Aplicando normalización...")
df_clean['NameNormalized'] = df_clean['Name'].apply(normalize_name)

normalized_unique_names = df_clean['NameNormalized'].nunique()
delta = original_unique_names - normalized_unique_names
print(f"✅ Nombres únicos (normalizados): {normalized_unique_names:,}")
print(f"📉 Reducción de duplicados por formato: {delta:,} nombres unificados")

# -------------------------
# Comparar ejemplos
# -------------------------
print("\n🔍 COMPARACIÓN DE NOMBRES:")
changed = df_clean[['Name', 'NameNormalized']].drop_duplicates()
changed = changed[changed['Name'] != changed['NameNormalized']].head(15)

if changed.empty:
    print("No se encontraron diferencias entre nombres originales y normalizados.")
else:
    for _, row in changed.iterrows():
        print(f"'{row['Name']}' → '{row['NameNormalized']}'")

# -------------------------
# Atletas con más competencias
# -------------------------
print("\n🏋️ TOP 10 ATLETAS CON MÁS COMPETENCIAS:")
athlete_counts = df_clean['NameNormalized'].value_counts().head(10)

for name, count in athlete_counts.items():
    atleta_df = df_clean[df_clean['NameNormalized'] == name]
    year_min = atleta_df['Year'].min()
    year_max = atleta_df['Year'].max()
    main_country = atleta_df['Country'].mode()[0] if not atleta_df['Country'].isna().all() else "Desconocido"
    print(f"🔹 {name}: {count:,} competencias ({year_min}–{year_max}) – {main_country}")


In [ ]:
# CELDA 9
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Limpieza de países, federaciones y agregado de continentes

print("🌍 NORMALIZACIÓN DE PAÍSES Y FEDERACIONES")
print("="*60)

# -------------------------
# Normalizar nombres de países
# -------------------------
print("🔁 Limpiando columna 'Country'...")
df_clean['Country'] = df_clean['Country'].fillna("Unknown").str.strip()

# Mostrar los más frecuentes
print("\n🌐 TOP 10 PAÍSES MÁS FRECUENTES:")
print(df_clean['Country'].value_counts().head(10))

# -------------------------
# Limpiar valores poco frecuentes o sospechosos
# -------------------------
rare_countries = df_clean['Country'].value_counts()[df_clean['Country'].value_counts() < 5]
df_clean['Country'] = df_clean['Country'].apply(lambda x: "Other" if x in rare_countries else x)

# -------------------------
# Normalizar nombre de federaciones
# -------------------------
print("\n🏢 TOP 10 FEDERACIONES MÁS FRECUENTES:")
df_clean['Federation'] = df_clean['Federation'].fillna("Unknown").str.strip()
print(df_clean['Federation'].value_counts().head(10))

# -------------------------
# Agregar columna 'Continent' desde país (con ayuda de pycountry_convert)
# -------------------------
try:
    import pycountry_convert as pc

    def country_to_continent(country):
        try:
            # Conversión país → código alpha-2 → continente
            country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
            continent_code = pc.country_alpha2_to_continent_code(country_code)
            continent_map = {
                "AF": "África",
                "AS": "Asia",
                "EU": "Europa",
                "NA": "América del Norte",
                "OC": "Oceanía",
                "SA": "América del Sur",
                "AN": "Antártida"
            }
            return continent_map.get(continent_code, "Desconocido")
        except:
            return "Desconocido"

    print("\n🗺️ Asignando continentes...")
    df_clean['Continent'] = df_clean['Country'].apply(country_to_continent)
    print("✅ Continentes asignados.")
    print(df_clean['Continent'].value_counts())

except ImportError:
    print("⚠️ No se encontró pycountry_convert. Ejecuta: pip install pycountry-convert")
    df_clean['Continent'] = "Desconocido"

print(f"\n✅ Total final de filas: {len(df_clean):,}")


In [ ]:
# CELDA 10
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Análisis de anomalías en edades y totales

print("🔎 INVESTIGANDO ANOMALÍAS EN LOS DATOS...")
print("="*60)

# -------------------------
# Niños con totales altos
# -------------------------
print("\n🚨 CASOS SOSPECHOSOS: <15 años con >300 kg")
young_heavy = df_clean[(df_clean['Age'] < 15) & (df_clean['TotalKg'] > 300)]

if young_heavy.empty:
    print("✅ No se encontraron casos extremos en esta categoría.")
else:
    print(f"🔍 {len(young_heavy)} casos detectados:\n")
    for _, row in young_heavy[['NameNormalized', 'Age', 'TotalKg', 'BodyweightKg', 'Country', 'Date', 'Federation']].head(10).iterrows():
        print(f"🔹 {row['NameNormalized']}: {row['Age']} años — {row['TotalKg']}kg (peso: {row['BodyweightKg']}kg) – {row['Country']}")

# -------------------------
# Resumen de distribución de edad
# -------------------------
print("\n📊 DISTRIBUCIÓN DE EDADES:")
print(df_clean['Age'].describe().round(1))

# Casos extremos
print("\n📍 CASOS EXTREMOS:")
print(f"👶 Menores de 10 años: {len(df_clean[df_clean['Age'] < 10]):,}")
print(f"👴 Mayores de 80 años: {len(df_clean[df_clean['Age'] > 80]):,}")

# -------------------------
# Análisis por grupo etario detallado
# -------------------------
print("\n📈 TOTALES PROMEDIO POR GRUPO DE EDAD:")
age_bins = [0, 12, 18, 25, 35, 45, 55, 65, 100]
age_labels = ['<12', '12-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
df_clean['AgeGroupDetailed'] = pd.cut(df_clean['Age'], bins=age_bins, labels=age_labels)

age_analysis = df_clean.groupby('AgeGroupDetailed')['TotalKg'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
print(age_analysis)

# -------------------------
# Casos realmente extremos (error probable)
# -------------------------
print("\n🧯 POSIBLES ERRORES (Ej. Niños con +200 kg o Adultos Mayores con +600 kg):")
extreme_cases = df_clean[
    ((df_clean['Age'] < 12) & (df_clean['TotalKg'] > 200)) |
    ((df_clean['Age'] > 80) & (df_clean['TotalKg'] > 600))
]

if extreme_cases.empty:
    print("✅ Sin casos extremos evidentes.")
else:
    for _, row in extreme_cases[['NameNormalized', 'Age', 'TotalKg', 'Date', 'Federation']].head(5).iterrows():
        print(f"⚠️ {row['NameNormalized']}: {row['Age']} años – {row['TotalKg']}kg ({row['Federation']})")


In [ ]:
# CELDA 11
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Análisis por clasificación etaria oficial (AgeClass)

print("🧬 ANALIZANDO DATOS CON CLASIFICACIONES DE EDAD OFICIALES")
print("=" * 60)

# -------------------------
# Conteo de categorías
# -------------------------
print("\n📚 CATEGORÍAS DE EDAD DISPONIBLES:")
age_class_counts = df_clean['AgeClass'].value_counts()
print(age_class_counts.head(15))

print(f"\n✅ Registros con AgeClass: {df_clean['AgeClass'].notna().sum():,}")
print(f"❌ Registros sin AgeClass: {df_clean['AgeClass'].isna().sum():,}")

# -------------------------
# Filtrar registros válidos
# -------------------------
df_clean_age = df_clean[df_clean['AgeClass'].notna()].copy()
print(f"\n📦 Dataset con AgeClass válido: {len(df_clean_age):,} registros")

# -------------------------
# Totales por AgeClass
# -------------------------
print("\n📊 TOTALES POR CATEGORÍA DE EDAD:")
age_class_stats = df_clean_age.groupby('AgeClass')['TotalKg'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
age_class_stats = age_class_stats.sort_values('count', ascending=False)
print(age_class_stats.head(10))

# -------------------------
# Totales por AgeClass y Sexo
# -------------------------
print("\n⚖️ TOTALES POR CATEGORÍA DE EDAD Y SEXO:")
age_sex_stats = df_clean_age.groupby(['AgeClass', 'Sex'])['TotalKg'].agg(['count', 'mean']).round(1)

if 'M' in df_clean_age['Sex'].unique():
    print("\n👨 CATEGORÍAS MASCULINAS MÁS COMUNES:")
    male_stats = age_sex_stats.xs('M', level='Sex').sort_values('count', ascending=False).head(8)
    print(male_stats)

if 'F' in df_clean_age['Sex'].unique():
    print("\n👩 CATEGORÍAS FEMENINAS MÁS COMUNES:")
    female_stats = age_sex_stats.xs('F', level='Sex').sort_values('count', ascending=False).head(8)
    print(female_stats)

# -------------------------
# Actualizar df_clean principal
# -------------------------
df_clean = df_clean_age.copy()
print(f"\n🔁 Dataset actualizado: {len(df_clean):,} registros con AgeClass")
# CELDA 12

In [ ]:
# CELDA 12
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Eliminación de registros con AgeClass vacíos o no realistas

print("🧹 ELIMINANDO CATEGORÍAS DE EDAD NO REALISTAS...")
print("=" * 60)

# Estado inicial
total_before = len(df_clean)
print(f"🔢 Registros iniciales: {total_before:,}")

# 1. Eliminar AgeClass == '5-12'
df_clean = df_clean[df_clean['AgeClass'] != '5-12']

# 2. Eliminar registros sin AgeClass
df_clean = df_clean[df_clean['AgeClass'].notna()]

# Estado final
total_after = len(df_clean)
removed = total_before - total_after
pct_removed = (removed / total_before) * 100

print(f"\n✅ Registros después del filtro: {total_after:,}")
print(f"❌ Eliminados: {removed:,} ({pct_removed:.2f}%)")

# Verificar categorías finales
print(f"\n📚 CATEGORÍAS RESTANTES:")
remaining_categories = df_clean['AgeClass'].value_counts().sort_index()
for cat, count in remaining_categories.items():
    avg_total = df_clean[df_clean['AgeClass'] == cat]['TotalKg'].mean()
    print(f"  {cat}: {count:,} registros – Promedio TotalKg: {avg_total:.1f}kg")

# Estadísticas de TotalKg
print(f"\n📊 ESTADÍSTICAS DE TOTALES (POST-FILTRO):")
print(df_clean['TotalKg'].describe().round(1))


In [ ]:
# CELDA 13
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Muestra aleatoria de 20 filas con todas las columnas

print("🔍 MUESTRA ALEATORIA DEL DATASET LIMPIO")
print("=" * 60)

# Verificación de columnas
print(f"🧩 Total columnas: {len(df_clean.columns)}")
print("📝 Columnas disponibles:")
for col in df_clean.columns:
    print(f"  - {col}")
print()

# Mostrar 20 filas aleatorias
print("📋 Muestra aleatoria de 20 filas:")
sample_df = df_clean.sample(n=20, random_state=42)  # Fijamos seed para reproducibilidad

try:
    display(sample_df)  # Solo si estás en Jupyter o Streamlit
except NameError:
    print(sample_df.to_string(index=False))


In [ ]:
# CELDA 14
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Limpieza avanzada y estandarización de campos

print("🧼 LIMPIEZA AVANZADA DE DATOS")
print("="*60)

# ------------------------------
# 1. Normalizar campos categóricos
# ------------------------------
print("🔠 Normalizando columnas categóricas...")

df_clean['Sex'] = df_clean['Sex'].str.upper().str.strip()
df_clean['Equipment'] = df_clean['Equipment'].str.title().str.strip()
df_clean['Tested'] = df_clean['Tested'].astype(str).str.upper().str.strip()
df_clean['Federation'] = df_clean['Federation'].astype(str).str.strip()
df_clean['Event'] = df_clean['Event'].astype(str).str.title().str.strip()

# ------------------------------
# 2. Corregir eventos inconsistentes
# ------------------------------
print("🎯 Corrigiendo valores en 'Event'...")

event_map = {
    'Sbd': 'Full Power',
    'Fullpower': 'Full Power',
    'Full Powerlifting': 'Full Power',
    'Bench': 'Bench Only',
    'Deadlift': 'Deadlift Only',
    'Push Pull': 'Push-Pull',
    'Pushpull': 'Push-Pull',
}
df_clean['Event'] = df_clean['Event'].replace(event_map)

# ------------------------------
# 3. Limpiar federaciones nulas o inválidas
# ------------------------------
print("🏛️ Normalizando federaciones...")

df_clean['Federation'] = df_clean['Federation'].replace(
    ['', '-', 'N/A', 'na', 'NaN', 'None'], 'Desconocida'
)

# ------------------------------
# 4. Detectar fechas en el futuro
# ------------------------------
print("🕓 Verificando fechas futuras...")

today = pd.Timestamp.now()
future_dates = df_clean[df_clean['Date'] > today]
print(f"🔮 Registros con fecha en el futuro: {len(future_dates)}")

# ------------------------------
# 5. Eliminar duplicados exactos
# ------------------------------
print("📎 Eliminando duplicados exactos por Nombre + Fecha + Federación...")

before_dupes = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['NameNormalized', 'Date', 'Federation'])
after_dupes = len(df_clean)
print(f"🗑️ Registros eliminados por duplicados: {before_dupes - after_dupes:,}")

# ------------------------------
# 6. Normalizar países (opcional manual)
# ------------------------------
print("🌍 Normalizando nombres de países...")

country_map = {
    'USA': 'United States',
    'UK': 'United Kingdom',
    'Korea': 'South Korea',
    'UAE': 'United Arab Emirates',
    'PR': 'Puerto Rico',
    'TR': 'Turkey',
    'IR': 'Iran',
    # Agrega más según lo que detectes
}
df_clean['Country'] = df_clean['Country'].replace(country_map)

# ------------------------------
# 7. Crear clasificación Raw vs Equipado simplificada
# ------------------------------
print("🏋️ Clasificando Raw vs Equipado...")

def classify_equipment(e):
    if pd.isna(e): return "Desconocido"
    e = str(e).lower()
    if "raw" in e: return "Raw"
    if "wraps" in e: return "Raw"
    if "single" in e or "multi" in e or "equipped" in e: return "Equipped"
    return "Otro"

df_clean['RawOrEquipped'] = df_clean['Equipment'].apply(classify_equipment)

# ------------------------------
# 8. Validación final de columnas
# ------------------------------
print("\n✅ LIMPIEZA AVANZADA COMPLETADA")
print(f"🧩 Columnas disponibles: {len(df_clean.columns)}")
print("🔎 Nuevas columnas agregadas: ['RawOrEquipped']")

# Opcional: ver una muestra final
print("\n📋 Muestra final tras limpieza:")
print(df_clean.sample(5)[['NameNormalized', 'Sex', 'Equipment', 'RawOrEquipped', 'Country', 'Event', 'Federation']])


In [ ]:
# CELDA 15
# -*- coding: utf-8 -*-
# ---
# Powerlifting Data Analysis
# Normalización y modelo estrella (dimensiones + hechos)

print("🔁 NORMALIZANDO Y CREANDO MODELO ESTRELLA")
print("="*60)

# -----------------------------
# Crear tabla DIM_MEET
# -----------------------------
print("🧱 Creando DIM_MEET...")
dim_meet = df_clean[['MeetName', 'MeetTown', 'MeetState', 'MeetCountry', 'Date']].drop_duplicates().reset_index(drop=True)
dim_meet['MeetID'] = range(1, len(dim_meet) + 1)
print(f"Dimensión MEET: {len(dim_meet):,} filas")

# -----------------------------
# Crear tabla DIM_FEDERATION
# -----------------------------
print("🧱 Creando DIM_FEDERATION...")
dim_fed = df_clean[['Federation', 'ParentFederation']].drop_duplicates().reset_index(drop=True)
dim_fed['FederationID'] = range(1, len(dim_fed) + 1)
print(f"Dimensión FEDERATION: {len(dim_fed):,} filas")

# -----------------------------
# Crear tabla DIM_ATLETA
# -----------------------------
print("🧱 Creando DIM_ATLETA...")
dim_atleta = df_clean[['NameNormalized', 'Sex', 'Age', 'AgeClass', 'AgeGroup', 'AgeGroupDetailed', 'BodyweightKg', 'WeightClass']].drop_duplicates().reset_index(drop=True)
dim_atleta['AthleteID'] = range(1, len(dim_atleta) + 1)
print(f"Dimensión ATLETA: {len(dim_atleta):,} filas")

# -----------------------------
# Reemplazar en tabla de hechos (df_fact)
# -----------------------------
print("📦 Construyendo tabla de hechos...")

# Merge federaciones
df_fact = df_clean.merge(dim_fed, on=['Federation', 'ParentFederation'], how='left')

# Merge meet
df_fact = df_fact.merge(dim_meet, on=['MeetName', 'MeetTown', 'MeetState', 'MeetCountry', 'Date'], how='left')

# Merge atleta
df_fact = df_fact.merge(dim_atleta, on=['NameNormalized', 'Sex', 'Age', 'AgeClass', 'AgeGroup', 'AgeGroupDetailed', 'BodyweightKg', 'WeightClass'], how='left')

# Seleccionar columnas finales
fact_cols = [
    'AthleteID', 'FederationID', 'MeetID',
    'Event', 'Equipment', 'Tested',
    'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg',
    'Dots', 'Wilks', 'Glossbrenner', 'Goodlift',
    'Year', 'Decade', 'RelativeStrength', 'Place'
]

df_fact = df_fact[fact_cols].copy()

# Mostrar estructura
print(f"\n📈 TABLA DE HECHOS: {len(df_fact):,} filas × {len(df_fact.columns)} columnas")
print("🧱 Listo para exportar a modelo estrella en GCP BigQuery o Snowflake 🚀")


In [ ]:
# CELDA 16 - Configuración segura para carga a BigQuery (.env en raíz del proyecto)
# -*- coding: utf-8 -*-

import os
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv
from pathlib import Path

# --------------------------
# Ruta base del proyecto
# --------------------------
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

# --------------------------
# Cargar variables desde el archivo .env en la raíz del proyecto
# --------------------------
load_dotenv(dotenv_path=BASE_DIR / ".env")

PROJECT_ID = os.getenv("PROJECT_ID")
DATASET_ID = os.getenv("DATASET_ID")
CREDENTIALS_PATH = os.getenv("CREDENTIALS_PATH")  # Asegúrate de usar esta clave en el .env

# --------------------------
# Crear cliente BigQuery
# --------------------------
if not all([PROJECT_ID, DATASET_ID, CREDENTIALS_PATH]):
    raise ValueError("❌ Faltan variables de entorno. Verifica tu archivo .env en la raíz del proyecto.")

credentials = service_account.Credentials.from_service_account_file(BASE_DIR / CREDENTIALS_PATH)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

print(f"✅ Cliente BigQuery creado con proyecto: {PROJECT_ID}")


In [ ]:
# CELDA 17 - Cargar df_clean a BigQuery
# -*- coding: utf-8 -*-

from google.cloud import bigquery

# --------------------------
# Verificar DataFrame
# --------------------------
if 'df_clean' not in locals():
    raise ValueError("❌ df_clean no está definido. Asegúrate de haber ejecutado el procesamiento anterior.")

# --------------------------
# Nombre completo de la tabla
# --------------------------
TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.results_clean"

print(f"📦 Preparando carga a tabla: {TABLE_ID}")
print(f"🔢 Filas: {len(df_clean):,} | Columnas: {len(df_clean.columns)}")

# --------------------------
# Cargar a BigQuery
# --------------------------
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=True
)

job = client.load_table_from_dataframe(df_clean, TABLE_ID, job_config=job_config)
job.result()  # Esperar a que termine

print("✅ Carga completada con éxito")
